In [11]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import linalg_ops
from tensorflow.python.ops import math_ops
import networkx as nx
import numpy as np



In [7]:
@ops.RegisterGradient("test1")
def _test1(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan"""
    e = op.outputs[0]
    v = op.outputs[1]
    #dim = v.get_shape()
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_grassman = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v, array_ops.transpose(grad_v)), v)
            grad_a = math_ops.batch_matmul(grad_grassman, math_ops.batch_matmul(E, array_ops.transpose(grad_v)))+math_ops.batch_matmul(grad_v, math_ops.batch_matmul(E, array_ops.transpose(grad_grassman)))
    return grad_a
                                                                 
@ops.RegisterGradient("test2adslfkas")
def _test2(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan with grad_stiefel instead of grassman"""
    e = op.outputs[0]
    v = op.outputs[1]
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_stiefel = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v, array_ops.transpose(v)), grad_v)
            grad_a = math_ops.batch_matmul(grad_stiefel, math_ops.batch_matmul(E, array_ops.transpose(grad_v)))+math_ops.batch_matmul(grad_v, math_ops.batch_matmul(E, array_ops.transpose(grad_stiefel)))
                                           
    return grad_a 

In [27]:
?nx.planted_partition_graph

def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=0.5, p_out=0.1, seed=None):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out, seed=seed)
    A = nx.adjacency_matrix(G).todense()
    
    return A




In [183]:
groupsize=10
communities=2
A =balanced_stochastic_blockmodel(communities = communities, groupsize = groupsize, seed=1, p_in=0.8, p_out=0.05)
r = np.sqrt(np.sum(A)/(2.0*groupsize*communities))
Diag = 2*np.identity(communities*groupsize)#np.reshape(np.sum(A,1), [6,1])
BH_op = (np.square(r)-1)*np.identity(communities*groupsize)-r*A+Diag
val, vec = np.linalg.eig(BH_op)
val, vec
proj = A*vec[:,0:2]


In [184]:
proj

matrix([[-2.25081171, -0.42134974],
        [-2.55006201, -0.56077068],
        [-2.40267423, -0.81077109],
        [-2.65278347, -0.86284988],
        [-2.37596603, -0.79651477],
        [-1.85343208, -0.60516954],
        [-2.49116716, -0.53669218],
        [-2.19672164, -0.73597218],
        [-2.65278347, -0.86284988],
        [-2.44683088, -0.78805098],
        [-0.62612423,  1.87837242],
        [-0.62518923,  1.87886105],
        [-0.74132535,  2.138195  ],
        [-0.9290094 ,  1.83883864],
        [-0.88037077,  2.60258874],
        [-0.92153002,  1.84230338],
        [-0.56562846,  1.8972771 ],
        [-0.77125962,  2.35226594],
        [-0.88037077,  2.60258874],
        [-0.98192558,  2.02526679]])

In [209]:
def proj_magnitude(space, vector):
    projection_op = tf.matmul(space, tf.transpose(space))
    projection = tf.matmul(projection_op, vector)
    return tf.reduce_sum(projection*projection)

vec = tf.ones([20, 1], dtype=tf.float64)
X = tf.Variable(np.identity(20))
init = tf.initialize_all_variables()

compare = tf.reduce_sum(vec*vec)

with tf.Session() as sess:
    sess.run(init)
    print tf.matmul(X, vec).eval(), proj_magnitude(X, vec).eval(), compare.eval(),  vec.eval()#print sess.run([proj_magnitude(proj, vec)])

[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]] 20.0 20.0 [[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]


In [238]:
def test_eigenloss(data=A, target = proj, communities = 2, group_size = 10, seed=1, p=1.0, q=0.0, name='test1', projection_dim=2):
    """testing to see if the loss will decrease backproping through very simple function"""
    #B = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=p, p_out=q, seed=seed)).astype(np.double)
    B = A
    B = tf.cast(B, dtype = tf.float64)
    
    Diag = tf.diag(tf.reduce_sum(B,0))
    Diag = tf.cast(Diag, tf.float64)

    #R =  tf.Variable(tf.random_normal(shape=[communities*group_size,communities*group_size], mean=0.0,
    #                             stddev=0.4, dtype=tf.float64,
    #                             seed=None, name=None))

    r = tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=0.4, dtype=tf.float64,
                                 seed=None, name=None))
    
    BH = r*B#(tf.square(r)-1)*tf.diag(tf.ones(shape=[communities*group_size], dtype=tf.float64))-tf.mul(r, B)+Diag 
    

    with tf.Session() as sess:
        g = tf.get_default_graph()
        
        with g.gradient_override_map({'SelfAdjointEigV2': name}):
            eigenval, eigenvec = tf.self_adjoint_eig(BH)
            eigenvec_proj = tf.slice(eigenvec, [0,0], [communities*group_size, projection_dim])
            
            target = proj
            
            rnd_vec1 = tf.Variable(tf.random_normal(shape=[communities*group_size,1], mean=0.0,stddev=0.4,
                                                    dtype=tf.float64,
                                                    seed=None, name=None))
            rnd_vec2 = tf.Variable(tf.random_normal(shape=[communities*group_size,1], mean=0.0,stddev=0.4,
                                                    dtype=tf.float64,
                                                    seed=None, name=None))
            
            vec1_mag_space = proj_magnitude(eigenvec_proj, rnd_vec1)
            vec1_mag_target = proj_magnitude(proj, rnd_vec1)
            
            vec2_mag_space = proj_magnitude(eigenvec_proj, rnd_vec2)
            vec2_mag_target = proj_magnitude(proj, rnd_vec2)
            
            diff_1 = tf.abs(vec1_mag_space-vec1_mag_target)
            diff_2 = tf.abs(vec2_mag_space-vec2_mag_target)
            
            loss = (diff_1+diff_2)/2
            optimizer = tf.train.GradientDescentOptimizer(0.01)
            train = optimizer.minimize(loss, var_list=[r])
            
            init = tf.initialize_all_variables()
            
            grad = tf.gradients(loss, r)
            
            sess.run(init)
            for i in range(100):
                #sess.run(init)
                if i%5:
                    print sess.run([train,r, loss, grad, vec1_mag_space, vec1_mag_target])
                sess.run([train])
                
                

In [239]:
test_eigenloss(p=0.8, q=0.05, seed=1)
#if they don't move when the random vectors are the same, if they aren't hte loss fluctuates 
#completely as a result of a change in the vectors


#r is exploding in value with no difference in the projections.... need to see whether you forgot to normalize.  


[None, array([-0.60509453]), 472.8446509539466, [array([ 5.85742834])], 0.18938780791481105, 766.8236364005179]
[None, array([-0.74174934]), 472.8446509539466, [array([ 7.18027246])], 0.18938780791481105, 766.8236364005179]
[None, array([-0.90926633]), 472.8446509539466, [array([ 8.80186827])], 0.18938780791481105, 766.8236364005179]
[None, array([-1.11461543]), 472.8446509539466, [array([ 10.78968597])], 0.18938780791481105, 766.8236364005179]
[None, array([-1.51278065]), 472.8446509539466, [array([ 14.64399982])], 0.18938780791481105, 766.8236364005179]
[None, array([-1.85442768]), 472.8446509539466, [array([ 17.95120701])], 0.18938780791481105, 766.8236364005179]
[None, array([-2.27323241]), 472.8446509539466, [array([ 22.00531528])], 0.18938780791481105, 766.8236364005179]
[None, array([-2.7866202]), 472.8446509539466, [array([ 26.9750051])], 0.18938780791481105, 766.8236364005179]
[None, array([-3.78206241]), 472.8446509539466, [array([ 36.61107202])], 0.18938780791481105, 766.823

#wow!  The loss did go down!  So in the case of having full control over what the other matrix is,
#we can make the eigendecomposition go down.  Out of curiousity, does this work with the original gradient, and with the steifel one?



In [56]:
test_eigenloss(name='test1') #stefiel test

[None, 20.153083393626535, array([[-1.6229856 , -0.59699278, -1.89746628, -5.13802015,  1.23753856,
         1.07648901],
       [ 0.17576   ,  0.25889974,  5.38365928, -1.9536869 , -0.05439934,
         0.32673333],
       [ 1.64577052, -1.38526026, -1.70834788,  0.58499732, -1.27315463,
        -1.8979476 ],
       [ 1.84324931, -0.31946612, -2.12173734, -1.02892922, -0.51431976,
        -3.53710599],
       [-1.58722988,  0.02551298,  0.77893913, -0.33201291, -2.44808229,
        -3.48726305],
       [ 1.20845794, -1.36937894,  0.85512335, -0.70545307,  1.22127079,
        -0.6373378 ]])]
[None, 19.992341748290844, array([[-1.6229856 , -0.5946072 , -1.89746628, -5.13802015,  1.23259336,
         1.07648901],
       [ 0.17505767,  0.25889974,  5.38365928, -1.9536869 , -0.05439934,
         0.32542771],
       [ 1.64577052, -1.38526026, -1.70834788,  0.58499732, -1.27315463,
        -1.8979476 ],
       [ 1.84324931, -0.31946612, -2.12173734, -1.02892922, -0.51431976,
        -3.53710

In [57]:
#also went down, how about original?

test_eigenloss(name='SelfAdjointEigV2')

#also works

[None, 23.087245390944911, array([[-1.401837  , -1.91498251,  1.49405516, -2.52822293,  0.08392844,
        -2.0596564 ],
       [ 0.92143784,  0.630483  , -3.84025062,  0.07877796, -3.36876601,
         0.17751974],
       [ 1.29082177, -1.4207878 ,  1.54161114,  1.33839961, -0.59905263,
         0.43587591],
       [-2.44281479, -0.98540297, -1.8615756 ,  2.15521821,  1.82751102,
        -0.10431622],
       [ 2.92911161, -0.04522269, -0.78113057, -1.18749077, -5.07933967,
         1.03494463],
       [ 1.25038735,  0.07304619, -0.34865746,  0.36595329,  2.96379569,
         1.22015062]])]
[None, 22.90310077456413, array([[-1.401837  , -1.90733024,  1.49405516, -2.52822293,  0.08359306,
        -2.0596564 ],
       [ 0.91775577,  0.630483  , -3.84025062,  0.07877796, -3.36876601,
         0.17681037],
       [ 1.29082177, -1.4207878 ,  1.54161114,  1.33839961, -0.59905263,
         0.43587591],
       [-2.44281479, -0.98540297, -1.8615756 ,  2.15521821,  1.82751102,
        -0.104316

In [58]:
#probably worth checking on tensorboard the comparison in behaviour

def test(communities = 2, group_size = 3, seed=1, p=0.5, q=0.3):
    """testing to see if the loss will decrease backproping through very simple function"""
    B = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=p, p_out=q, seed=seed)).astype(np.double)
    B = tf.cast(B, dtype = tf.float64)
    
    Diag = tf.diag(tf.reduce_sum(B,0))
    Diag = tf.cast(Diag, tf.float64)

    R =  tf.Variable(tf.random_normal(shape=[6,6], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

    BH = R*B#(tf.square(r)-1)*tf.diag(tf.ones(shape=[communities*group_size], dtype=tf.float64))-tf.mul(r, B)+Diag 
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        sess.run(init)
        eigenval, eigenvec = tf.self_adjoint_eig(BH)
        loss = tf.reduce_sum(tf.square(BH)) 
        optimizer = tf.train.GradientDescentOptimizer(.001)
        train = optimizer.minimize(loss, var_list=[R])
                  
        for i in range(1000):
            if i%500:
                print sess.run([train,loss, R ])
                sess.run([train])

In [59]:
test()

[None, 25.239922045716504, array([[ 0.32359606, -0.76596072, -4.63248471,  0.05573183,  1.39152726,
         0.41231821],
       [-0.13962645, -0.07660691, -1.69190653, -1.4381463 ,  2.10993716,
         1.456059  ],
       [-0.28289719,  3.88469375, -0.05370114,  2.06877643,  3.36157238,
        -1.39207394],
       [-3.44590236, -1.82799632, -0.54536673,  3.92033976, -1.82300007,
        -3.42372844],
       [-0.97964951, -0.54193757,  2.6134948 ,  2.59477656, -7.730212  ,
         3.90788917],
       [-2.06167889,  1.04636129,  2.83020911,  0.0094912 ,  1.77488961,
        -0.55341839]])]
[None, 25.038607610672976, array([[ 0.32359606, -0.76289994, -4.63248471,  0.05573183,  1.38596672,
         0.41231821],
       [-0.1390685 , -0.07660691, -1.69190653, -1.4381463 ,  2.10993716,
         1.45024059],
       [-0.28289719,  3.88469375, -0.05370114,  2.06877643,  3.36157238,
        -1.39207394],
       [-3.44590236, -1.82799632, -0.54536673,  3.92033976, -1.82300007,
        -3.42372

In [60]:
test_eigenloss(p=0.9, q=0.0)

[None, 35.993977115424826, array([[-0.96718893,  1.58378194,  0.66938876,  1.61018085, -0.92622857,
         0.00912788],
       [ 1.6353203 , -0.72117736, -0.32308339,  1.922434  ,  0.57135188,
        -1.47062239],
       [-0.70904707,  2.5642724 ,  0.21130035,  3.04746752,  0.21416749,
        -0.98283784],
       [ 1.37161923,  1.85568264, -0.48432506, -0.03894419, -2.5374685 ,
         2.9619785 ],
       [ 0.62109847,  3.42268102,  2.22233551, -1.39732783, -0.46303653,
        -0.90495347],
       [ 0.88740731, -2.88332121, -0.48177442, -0.84595764,  2.08269763,
        -1.00272678]])]
[None, 35.706887989125761, array([[-0.96718893,  1.57745314,  0.66671388,  1.61018085, -0.92622857,
         0.00912788],
       [ 1.62878556, -0.72117736, -0.32179234,  1.922434  ,  0.57135188,
        -1.47062239],
       [-0.70621371,  2.55402557,  0.21130035,  3.04746752,  0.21416749,
        -0.98283784],
       [ 1.37161923,  1.85568264, -0.48432506, -0.03894419, -2.52732878,
         2.95014

In [ ]:
#ok, now implement subspace finding:

def test_eigenloss(communities = 2, group_size = 3, seed=1, p=0.5, q=0.3, name='test1', projection_dim=-2):
    """testing to see if the loss will decrease backproping through very simple function"""
    B = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=p, p_out=q, seed=seed)).astype(np.double)
    B = tf.cast(B, dtype = tf.float64)
    
    Diag = tf.diag(tf.reduce_sum(B,0))
    Diag = tf.cast(Diag, tf.float64)

    r =  tf.Variable(tf.random_normal(shape=[6,6], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

    BH = (tf.square(r)-1)*tf.diag(tf.ones(shape=[communities*group_size], dtype=tf.float64))-tf.mul(r, B)+Diag 
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        g = tf.get_default_graph()
        sess.run(init)
        with g.gradient_override_map({'SelfAdjointEigV2': name}):
            eigenval, eigenvec = tf.self_adjoint_eig(BH)
            
            eigen_proj = tf.slice(eigenvec, [0,0], [communities*group_size, projection_dim])
        
            loss = tf.reduce_sum(tf.square(eigenproj)) 
            optimizer = tf.train.GradientDescentOptimizer(.001)
            train = optimizer.minimize(loss, var_list=[R])
                  
            for i in range(1000):
                if i%500:
                    print sess.run([train,loss,R ])
                sess.run([train])
                
                
